In [1]:
pip install pymongo

StatementMeta(mySparkPool, 0, 2, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 58.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.functions import col, explode,current_date, datediff,from_unixtime, expr, avg
from pyspark.sql.types import IntegerType
from datetime import datetime


StatementMeta(mySparkPool, 12, 2, Finished, Available)

In [2]:

%%pyspark
df_accounts = spark.read.load('abfss://synapse@jgastorage.dfs.core.windows.net/accounts.json', format='json')

df_accounts.printSchema()
# ?? WHich is the most used Service ?? #

# unnesting the products array
df_exploded = df_accounts.select("account_id", "limit", explode(col("products")).alias("product"))

df_exploded = df_exploded.select(col("account_id.$numberInt").alias("account_id"), col("limit.$numberInt").alias("limit"), "product")



df_exploded.printSchema()


StatementMeta(mySparkPool, 12, 3, Finished, Available)

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- account_id: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- limit: struct (nullable = true)
 |    |-- $numberInt: string (nullable = true)
 |-- products: array (nullable = true)
 |    |-- element: string (containsNull = true)

root
 |-- account_id: string (nullable = true)
 |-- limit: string (nullable = true)
 |-- product: string (nullable = true)



In [32]:



df_exploded.write.mode("overwrite").saveAsTable("accounts")

df_exploded.write \
    .format("com.databricks.spark.csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save('abfss://synapse@jgastorage.dfs.core.windows.net/accounts.csv')

StatementMeta(mySparkPool, 5, 8, Finished, Available)

In [3]:

%%pyspark
df_customers = spark.read.load('abfss://synapse@jgastorage.dfs.core.windows.net/customers.json', format='json')

df_customers.show()

import math

#def calculate_age(seconds):
    #birthdate = datetime.fromtimestamp(seconds)
    #current_date = datetime.now()
   # age = current_date.year - birthdate.year - ((current_date.month, current_date.day) < (birthdate.month, birthdate.day))
    #return age

# ?? Wie viele Accounts hat jeder Kunde ?? #


def unix_timestamp_to_date(unix_timestamp):
    a = unix_timestamp / 1000
    return from_unixtime(a, "yyyy-MM-dd")

#calculate_age_udf = spark.udf.register("calculate_age", calculate_age, IntegerType())

spark.udf.register("unix_timestamp_to_date", unix_timestamp_to_date)



df_neu = df_customers.select("username", "name", "address",  col("birthdate.$date").alias("Time") , explode(col("accounts")).alias("account") )



df_ohne = df_neu.select("username", "name", "address", col("Time.$numberLong").alias("Time"), col("account.$numberInt").alias("account_id"))

df_ohne = df_ohne.withColumn("Time", col("Time").cast("bigint"))

df_ohne = df_ohne.withColumn("date_of_birth", unix_timestamp_to_date(col("Time")))
df_fertig = df_ohne.withColumn("age", datediff(current_date(), col("date_of_birth")) / 365)

#df_fertig = df_ohne.withColumn("age", calculate_age_udf(col("Time")) )

df_fertig.printSchema()

#df_fertig.write.mode("overwrite").saveAsTable("customers")

df_fertig.show()


df_fertig.write.mode("overwrite").saveAsTable("customers")

df_fertig.write \
    .format("com.databricks.spark.csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save('abfss://synapse@jgastorage.dfs.core.windows.net/customers.csv')

StatementMeta(mySparkPool, 12, 4, Finished, Available)

+--------------------+--------------------+------+--------------------+----------------+--------------------+-----------------+--------------------+----------------+
|                 _id|            accounts|active|             address|       birthdate|               email|             name|    tier_and_details|        username|
+--------------------+--------------------+------+--------------------+----------------+--------------------+-----------------+--------------------+----------------+
|{5ca4bbcea2dd94ee...|[{371138}, {32428...|  true|9286 Bethany Glen...|{{226117231000}}|arroyocolton@gmai...|    Elizabeth Ray|{null, null, null...|         fmiller|
|{5ca4bbcea2dd94ee...|          [{116508}]|  null|Unit 1047 Box 408...|{{761701587000}}|cooperalexis@hotm...|    Lindsay Cowan|{null, null, null...|valenciajennifer|
|{5ca4bbcea2dd94ee...|[{462501}, {22829...|  null|55711 Janet Plaza...|{{582848134000}}|timothy78@hotmail...|  Katherine David|{null, null, null...|      hillrachel|
|{5c

In [28]:
%%pyspark
df_transactions = spark.read.load('abfss://synapse@jgastorage.dfs.core.windows.net/transactions.json', format='json')


def unix_timestamp_to_date(unix_timestamp):
    a = unix_timestamp / 1000
    return from_unixtime(a, "yyyy-MM-dd")

spark.udf.register("unix_timestamp_to_date", unix_timestamp_to_date)

df_neu = df_transactions.select(col("account_id.$numberInt").alias("account_id"), col("bucket_end_date.$date.$numberLong").alias("end_date"), col("bucket_start_date.$date.$numberLong").alias("start_date"),
col("transaction_count.$numberInt").alias("transaction_count") , explode(col("transactions")).alias("transactions")        )


# calculating the lenght of the timebucket
df_neu = df_neu.withColumn("start_date", unix_timestamp_to_date(col("start_date")))

df_neu = df_neu.withColumn("end_date", unix_timestamp_to_date(col("end_date")))

df_neu = df_neu.withColumn("bucket_time", datediff(col("end_date"), col("start_date"))/365)


# now extracting the transaction_code and date

df_neu = df_neu.withColumn("transaction_code", col("transactions.transaction_code"))
df_neu = df_neu.withColumn("transaction_date", col("transactions.date.$date.$numberLong"))

# converting unix timestamp to date
df_neu = df_neu.withColumn("transaction_date", unix_timestamp_to_date(col("transaction_date")))

# now only select the necessary columns

df_neu = df_neu.select("account_id", "end_date", "start_date", "transaction_count", "bucket_time", "transaction_code", "transaction_date")

df_neu.printSchema()
df_neu.show()

# now save as table

df_neu.write.mode("overwrite").saveAsTable("transactions")

df_neu.write \
    .format("com.databricks.spark.csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save('abfss://synapse@jgastorage.dfs.core.windows.net/transactions.csv')


StatementMeta(mySparkPool, 5, 4, Finished, Available)

root
 |-- account_id: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- transaction_count: string (nullable = true)
 |-- bucket_time: double (nullable = true)
 |-- transaction_code: string (nullable = true)
 |-- transaction_date: string (nullable = true)

+----------+----------+----------+-----------------+------------------+----------------+----------------+
|account_id|  end_date|start_date|transaction_count|       bucket_time|transaction_code|transaction_date|
+----------+----------+----------+-----------------+------------------+----------------+----------------+
|    443178|2017-01-03|1969-02-04|               66|47.945205479452056|             buy|      2003-09-09|
|    443178|2017-01-03|1969-02-04|               66|47.945205479452056|             buy|      2016-06-14|
|    443178|2017-01-03|1969-02-04|               66|47.945205479452056|             buy|      2002-12-04|
|    443178|2017-01-03|1969-02-04|            

In [ ]:
# now vizualize the data

# Frage: Welcher Account typy ist der häufigste